### [Optical_flow](https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html)
Now let's discuss an important concept, "Optical Flow", which is related to videos and has many applications.

#### Goal
In this chapter,

* We will understand the concepts of optical flow and its estimation using Lucas-Kanade method.
* We will use functions like [cv.calcOpticalFlowPyrLK()](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323) to track feature points in a video.

#### Optical Flow
광학 흐름은 물체 또는 카메라의 이동에 의해 발생하는 두 개의 연속적인 프레임 사이의 이미지 물체의 외관상 모션 패턴입니다. 2D 벡터 필드이며 각 벡터는 첫 번째 프레임에서 두 번째 프레임까지의 점 이동을 보여주는 변위 벡터입니다. 아래 이미지를 살펴보십시오 (이미지 제공 : Optical Flow에 대한 Wikipedia 기사 ).
![](optical_flow_basic1.jpg)

그것은 5 개의 연속 된 프레임에서 움직이는 공을 보여줍니다. 화살표는 변위 벡터를 보여줍니다. 광 흐름은 다음과 같은 분야에서 많은 응용 분야를 가지고 있습니다 :

* 모션의 구조
* 비디오 압축
* 비디오 안정화 ...

옵티컬 플로우는 몇 가지 가정하에 작동합니다.

1. 객체의 픽셀 강도는 연속 프레임간에 변경되지 않습니다.
2. 인접한 픽셀은 비슷한 동작을합니다.

첫 번째 프레임의 픽셀 \\(I(x,y,t)\\)를 고려하십시오 (새 차원을 확인하십시오, 시간은 여기에 추가됩니다. 이전에는 이미지 만 사용 했으므로 시간이 필요하지 않았습니다). \\(dt\\) 시간 후에 찍은 다음 프레임에서 거리 \\((dx, dy)\\)만큼 이동합니다. 그래서 그 픽셀들이 동일하고 강도가 변하지 않기 때문에 우리는 말할 수 있습니다,

\\(I(x,y,t) = I(x+dx, y+dy, t+dt)\\)

그런 다음 오른쪽의 테일러 급수 근사법을 취하고 일반적인 용어를 제거하고 \\(dt\\)로 나눠 다음 방정식을 얻습니다.

\\(f_x u + f_y v + f_t = 0 \\)

\\(f_x = \frac{\partial f}{\partial x} \; ; \; f_y = \frac{\partial f}{\partial y}\\)

\\(u = \frac{dx}{dt} \; ; \; v = \frac{dy}{dt}\\)


위의 방정식을 옵티컬 플로 방정식이라고합니다. 그것 안에서 우리는 \\(f_x\\)와 \\(f_y\\)를 찾을 수 있습니다. 그것들은 이미지 그라디언트입니다. 마찬가지로 \\(f_t\\)는 시간에 따른 그래디언트입니다. 그러나 \\((u, v)\\)는 알려져 있지 않습니다. 이 방정식은 두 개의 미지 변수로 풀 수는 없습니다. 따라서이 문제를 해결하기 위해 여러 가지 방법이 제공되며 그 중 하나는 Lucas-Kanade입니다.

##### Lucas-Kanade method

우리는 이전에 모든 이웃 픽셀들이 비슷한 움직임을 보일 것이라는 가정을 보았습니다. 루카스 - 카나데 (Lucas-Kanade) 방법은 그 지점을 중심으로 3x3 패치를 사용합니다. 그래서 모든 9 점은 같은 동작을합니다. 우리는이 9 점에 대해 \\((f_x, f_y, f_t)\\)를 찾을 수 있습니다. 이제 우리의 문제는 지나치게 결정되는 두 개의 알려지지 않은 변수를 가진 9 개의 방정식을 풀어냅니다. 최소 제곱합 방법을 사용하면 더 나은 솔루션을 얻을 수 있습니다. 아래는 최종 방정식 2 방정식 2 알려지지 않은 문제이며 해답을 얻는 것입니다.

\\(\begin{bmatrix} u \\ v \end{bmatrix} = \begin{bmatrix} \sum_{i}{f_{x_i}}^2 & \sum_{i}{f_{x_i} f_{y_i} } \\ \sum_{i}{f_{x_i} f_{y_i}} & \sum_{i}{f_{y_i}}^2 \end{bmatrix}^{-1} \begin{bmatrix} - \sum_{i}{f_{x_i} f_{t_i}} \\ - \sum_{i}{f_{y_i} f_{t_i}} \end{bmatrix}\\)

(해리스 코너 탐지기와 역행렬의 유사성을 확인하십시오. 이것은 모서리가 추적되는 더 좋은 점임을 나타냅니다.)

그래서 사용자 관점에서 볼 때, 아이디어는 간단하고, 우리는 추적 할 몇 가지 포인트를주고, 우리는 그 포인트의 옵티컬 플로우 벡터를받 습니다. 그러나 다시 몇 가지 문제가 있습니다. 지금까지 우리는 작은 동작을 다루고 있었습니다. 그래서 큰 움직임이있을 때 실패합니다. 그래서 다시 피라미드로 갑니다. 피라미드에 올라가면 작은 동작이 제거되고 큰 동작은 작은 동작이 됩니다. Lucas-Kanade를 적용하면 규모와 함께 옵티컬 플로우가 생성됩니다.

#### Lucas-Kanade Optical Flow in OpenCV
OpenCV는이 모든 것을 하나의 함수 cv.calcOpticalFlowPyrLK ()에 제공 합니다. 여기에서는 비디오의 일부 지점을 추적하는 간단한 응용 프로그램을 만듭니다. 포인트를 결정하기 위해 우리는 cv.goodFeaturesToTrack ()을 사용 합니다. 첫 번째 프레임을 가져 와서 Shi-Tomasi 구석 점을 찾아낸 다음 Lucas-Kanade 옵티컬 플로우를 사용하여 점을 반복적으로 추적합니다. 함수 cv.calcOpticalFlowPyrLK ()에 대해 우리는 이전 프레임, 이전 포인트 및 다음 프레임을 전달합니다. 다음 점이 발견되면 1의 값을 갖는 일부 상태 번호와 함께 다음 점을 반환하고, 그렇지 않으면 0을 반환합니다. 우리는 다음 단계에서 이전 지점으로이 다음 지점을 반복적으로 전달합니다. 아래 코드를 참조하십시오.

In [31]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('slow.flv')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray       = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0             = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame   = cap.read()
    if not ret:
        break
        
    frame_gray  = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
   
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    good_new    = p1[st == 1]
    good_old    = p0[st == 1]
    # draw the tracks
    for i, (new ,old) in enumerate(zip(good_new, good_old)):
        a, b  = new.ravel()
        c, d  = old.ravel()
        mask  = cv.line(mask, (a, b), (c, d), color[i].tolist(), 1)
        frame = cv.circle(frame, (a, b), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0       = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()
cv.waitKey(1)
cap.release()

(이 코드는 다음 키포인트가 얼마나 정확한지 확인하지 않습니다. 따라서 이미지에서 어떤 피쳐 포인트가 사라지더라도 옵티컬 플로우가 다음 포인트를 찾을 가능성이 있습니다. OpenCV 샘플은 매 5 프레임마다 특징점을 찾는 샘플을 제공하며, 좋은 점만 선택하도록 옵티컬 플로 포인트의 역방향 체크를 실행합니다. 샘플 / 파이썬 / lk_track.py).

#### Dense Optical Flow in OpenCV
루카스 - 카나 데 (Lucas-Kanade) 방법은 희소 한 특징 집합 (이 예에서는 Shi-Tomasi 알고리즘을 사용하여 감지 된 모서리)에 대한 옵티컬 플로우를 계산합니다. OpenCV는 밀도가 높은 옵티컬 플로우를 찾는 또 다른 알고리즘을 제공합니다. 프레임의 모든 점에 대한 옵티컬 플로우를 계산합니다. 2003 년 Gunner Farneback의 "다항식 확장에 기반한 2 프레임 모션 추정"에서 설명하는 Gunner Farneback의 알고리즘을 기반으로합니다.

아래 샘플은 위의 알고리즘을 사용하여 밀도가 높은 옵티컬 플로우를 찾는 방법을 보여줍니다. 옵티컬 플로우 벡터 \\((u, v)\\)를 갖는 2 채널 어레이를 얻습니다. 우리는 그들의 규모와 방향을 찾습니다. 더 나은 시각화를 위해 결과의 색을 지정합니다. 방향은 이미지의 색조 값에 해당합니다. 크기는 값 평면에 해당합니다. 아래 코드를 참조하십시오.

In [25]:
import cv2 as cv
import numpy as np

cap         = cv.VideoCapture("vtest.avi")
ret, frame1 = cap.read()
prvs        = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv         = np.zeros_like(frame1)
hsv[...,1]  = 255

while(1):
    ret, frame2 = cap.read()
    next        = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow        = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang    = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[...,0]  = ang * 180 /np.pi / 2
    hsv[...,2]  = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr         = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cap.release()
cv.destroyAllWindows()
cv.waitKey(1)

-1

OpenCV comes with a more advanced sample on dense optical flow, please see samples/python/opt_flow.py.

#### Additional Resources

#### Exercises
Check the code in samples/python/lk_track.py. Try to understand the code.
Check the code in samples/python/opt_flow.py. Try to understand the code.

In [18]:
# %load ../python/opt_flow.py
#!/usr/bin/env python

'''
example to show optical flow

USAGE: opt_flow.py [<video_source>]

Keys:
 1 - toggle HSV flow visualization
 2 - toggle glitch

Keys:
    ESC    - exit
'''

# Python 2/3 compatibility
from __future__ import print_function
import sys
sys.path.insert(0, '../python')

import numpy as np
import cv2 as cv
import video


def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    cv.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis


def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bgr


def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv.remap(img, flow, None, cv.INTER_LINEAR)
    return res

if __name__ == '__main__':
    import sys
    print(__doc__)
    try:
        #fn = sys.argv[1]
        fn = 'slow.flv'
    except IndexError:
        fn = 0

    cam = video.create_capture(fn)
    ret, prev = cam.read()
    prevgray = cv.cvtColor(prev, cv.COLOR_BGR2GRAY)
    show_hsv = False
    show_glitch = False
    cur_glitch = prev.copy()

    while True:
        ret, img = cam.read()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevgray = gray

        cv.imshow('flow', draw_flow(gray, flow))
        if show_hsv:
            cv.imshow('flow HSV', draw_hsv(flow))
        if show_glitch:
            cur_glitch = warp_flow(cur_glitch, flow)
            cv.imshow('glitch', cur_glitch)

        ch = cv.waitKey(5)
        if ch == 27:
            break
        if ch == ord('1'):
            show_hsv = not show_hsv
            print('HSV flow visualization is', ['off', 'on'][show_hsv])
        if ch == ord('2'):
            show_glitch = not show_glitch
            if show_glitch:
                cur_glitch = img.copy()
            print('glitch is', ['off', 'on'][show_glitch])
    cv.destroyAllWindows()



example to show optical flow

USAGE: opt_flow.py [<video_source>]

Keys:
 1 - toggle HSV flow visualization
 2 - toggle glitch

Keys:
    ESC    - exit



In [12]:
# %load ../python/lk_track.py
#!/usr/bin/env python

'''
Lucas-Kanade tracker
====================

Lucas-Kanade sparse optical flow demo. Uses goodFeaturesToTrack
for track initialization and back-tracking for match verification
between frames.

Usage
-----
lk_track.py [<video_source>]


Keys
----
ESC - exit
'''

# Python 2/3 compatibility
import sys
sys.path.insert(0, '../python')
from __future__ import print_function

import numpy as np
import cv2 as cv
import video
from common import anorm2, draw_str
from time import clock

lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict( maxCorners = 500,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

class App:
    def __init__(self, video_src):
        self.track_len = 5
        self.detect_interval = 5
        self.tracks = []
        self.cam = video.create_capture(video_src)
        self.frame_idx = 0

    def run(self):
        while True:
            _ret, frame = self.cam.read()
            
            if not _ret:
                break
            
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            vis = frame.copy()

            if len(self.tracks) > 0:
                img0, img1 = self.prev_gray, frame_gray
                p0 = np.float32([tr[-1] for tr in self.tracks]).reshape(-1, 1, 2)
                p1, _st, _err = cv.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
                p0r, _st, _err = cv.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
                d = abs(p0-p0r).reshape(-1, 2).max(-1)
                good = d < 1
                new_tracks = []
                for tr, (x, y), good_flag in zip(self.tracks, p1.reshape(-1, 2), good):
                    if not good_flag:
                        continue
                    tr.append((x, y))
                    if len(tr) > self.track_len:
                        del tr[0]
                    new_tracks.append(tr)
                    cv.circle(vis, (x, y), 2, (0, 255, 0), -1)
                self.tracks = new_tracks
                cv.polylines(vis, [np.int32(tr) for tr in self.tracks], False, (0, 255, 0))
                draw_str(vis, (20, 20), 'track count: %d' % len(self.tracks))

            if self.frame_idx % self.detect_interval == 0:
                mask = np.zeros_like(frame_gray)
                mask[:] = 255
                for x, y in [np.int32(tr[-1]) for tr in self.tracks]:
                    cv.circle(mask, (x, y), 5, 0, -1)
                p = cv.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
                if p is not None:
                    for x, y in np.float32(p).reshape(-1, 2):
                        self.tracks.append([(x, y)])


            self.frame_idx += 1
            self.prev_gray = frame_gray
            cv.imshow('lk_track', vis)

            ch = cv.waitKey(100)
            if ch == 27:
                break

def main():
    import sys
    try:
        #video_src = sys.argv[1]
        #video_src = 'slow.flv'
        video_src = 'vtest.avi'
    except:
        video_src = 0

    print(__doc__)
    App(video_src).run()
    cv.destroyAllWindows()

if __name__ == '__main__':
    main()



Lucas-Kanade tracker

Lucas-Kanade sparse optical flow demo. Uses goodFeaturesToTrack
for track initialization and back-tracking for match verification
between frames.

Usage
-----
lk_track.py [<video_source>]


Keys
----
ESC - exit

